In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import plotly.graph_objects as go
import streamlit as st
import requests
import zipfile
from io import BytesIO
import re
from typing import List, Dict, Tuple
import openpyxl # Necesario para pandas engine='openpyxl'
import time # <-- Importado para los reintentos

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN Y CONTEXTO
# -----------------------------------------------------------------------------
# Recuperar variables inyectadas por app.py
palette = locals().get("active_palette", ["#889064", "#5da5da", "#1f77b4"])
active_font = locals().get("active_font", "sans-serif")

# Parámetros
DOWNLOAD_URL = "https://datatur.sectur.gob.mx/Documentos%20compartidos/CUADRO_DGAC.zip"
TITLE_TEXT = "Participación en el mercado (Enero–Septiembre 2025)"
NETWORK_TIMEOUT = 60 # Tiempo de espera extendido a 60 segundos
MAX_DOWNLOAD_RETRIES = 3 # Número máximo de intentos

# -----------------------------------------------------------------------------
# 2. UTILIDADES Y LÓGICA DE COLOR
# -----------------------------------------------------------------------------

def es_percent(x, dec=1):
    """Convierte proporción (0-1) a porcentaje con coma decimal."""
    if pd.isna(x): return ""
    s = f"{x*100:,.{dec}f}%"
    return s.replace(",", "X").replace(".", ",").replace("X", ".")

def build_color_map(names: List[str], palette: List[str]) -> Dict[str, str]:
    """Asigna colores de la paleta a las aerolíneas, priorizando las mayores."""
    p = (palette * 3)

    # Reglas dinámicas: Asignación prioritaria a las aerolíneas principales
    dynamic_rules = [
        ("Viva", p[0]),           # Color 1 (verde)
        ("Volaris", p[1]),        # Color 2 (azul claro)
        ("Aeroméxico", p[2]),     # Color 3 (azul medio)
        ("Aeroméxico Connect", p[2]), # Mismo color que AM
        ("Magnicharters", "#a6a6a6"), # Gris (mantener un color distinto para minorías)
        ("Mexicana", p[3] if len(p)>3 else "#8da0cb")
    ]

    cmap = {}
    for name in names:
        lname = str(name).lower()
        assigned = False
        for key, color in dynamic_rules:
            if key.lower() in lname:
                cmap[name] = color
                assigned = True
                break
        if not assigned:
            # Fallback para el resto (usar un gris estándar)
            cmap[name] = "#b0b0b0"
    return cmap

# -------------------------------------------------------------
# 3. CARGA DE DATOS (CACHED) - Operación en memoria con reintentos
# -------------------------------------------------------------

@st.cache_data(show_spinner="Descargando y extrayendo datos de participación de aerolíneas...")
def load_and_parse_data(url: str, sheet_name: str = "Nacionales") -> pd.DataFrame:
    """Descarga ZIP, extrae el Excel, y parsea la tabla de Aerolíneas."""

    # 1. Descarga del contenido del ZIP en memoria con reintentos
    response = None
    for attempt in range(MAX_DOWNLOAD_RETRIES):
        try:
            response = requests.get(url, allow_redirects=True, timeout=NETWORK_TIMEOUT)
            response.raise_for_status()
            break # Éxito en la descarga
        except requests.exceptions.Timeout:
            if attempt < MAX_DOWNLOAD_RETRIES - 1:
                time.sleep(2 * (attempt + 1)) # Espera 2s, luego 4s, etc.
                continue
            else:
                # Relanza el error en el último intento
                raise
        except Exception:
             # Para otros errores HTTP (4xx, 5xx) o de conexión que no sean solo timeout
             raise

    if response is None:
        raise RuntimeError("Fallo al descargar después de múltiples reintentos.")

    # 2. Abrir el contenido como un archivo ZIP en memoria
    with zipfile.ZipFile(BytesIO(response.content), 'r') as zf:
        xlsx_files = [name for name in zf.namelist() if name.lower().endswith(('.xlsx', '.xls'))]
        if not xlsx_files:
            raise RuntimeError("No se encontró un archivo Excel dentro del ZIP.")
        file_to_extract = xlsx_files[0]

        # 3. Leer el Excel directamente desde el ZIP
        xls_bytes = zf.read(file_to_extract)
        # Nota: Usamos header=None porque el bloque de datos no tiene encabezado estándar
        nat = pd.read_excel(BytesIO(xls_bytes), sheet_name=sheet_name, engine="openpyxl", header=None)

    # 4. Detectar y extraer el bloque de datos (parsing de encabezados apilados)
    start_idx = None
    for i in range(len(nat)):
        val = str(nat.iloc[i, 2])
        # Buscamos la fila donde el nombre de la aerolínea comienza a ser legible (col 2)
        if val and val != "nan" and "Aerolínea" not in val and not isinstance(nat.iloc[i, 3], str):
            start_idx = i
            break

    if start_idx is None:
        raise RuntimeError(f"No se pudo detectar el inicio de la tabla de aerolíneas en la hoja '{sheet_name}'.")

    end_idx = len(nat)
    for i in range(start_idx, len(nat)):
        val = str(nat.iloc[i, 2])
        if val.strip().lower() in ("nan", "", "t o t a l", "total"):
            end_idx = i
            break

    # Se asume la estructura fija de columnas 'Unnamed'
    rows = nat.iloc[start_idx:end_idx].copy()

    # 5. Mapear a nombres de columna limpios
    df = pd.DataFrame({
        "Aerolínea":           rows.iloc[:, 2],
        "Ene-Sep 2024":        rows.iloc[:, 3],
        "Ene-Sep 2025":        rows.iloc[:, 4],
        "Var Ene-Sep":         rows.iloc[:, 5],
        "Part Ene-Sep 2025":   rows.iloc[:, 6],
        "Septiembre 2024":     rows.iloc[:, 7],
        "Septiembre 2025":     rows.iloc[:, 8],
        "Var Sept":            rows.iloc[:, 9],
        "Part Sept 2025":      rows.iloc[:, 10],
    })
    df.columns = [c.strip() for c in df.columns]

    # 6. Limpieza y conversión
    df["Aerolínea"] = (
        df["Aerolínea"].astype(str)
        .str.replace(r"\)\d+$", ")", regex=True)
        .str.replace(r"\s+1$", "", regex=True)
        .str.strip()
    )

    for c in df.columns:
        if c != "Aerolínea":
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # Filtrar solo el DF necesario para el Pie
    pie_df = df[["Aerolínea", "Part Ene-Sep 2025"]].dropna().copy()
    pie_df = pie_df[pie_df["Part Ene-Sep 2025"] > 0]
    pie_df = pie_df.sort_values("Part Ene-Sep 2025", ascending=False).reset_index(drop=True)

    return pie_df

# -------------------------------------------------------------
# 4. FUNCIÓN PRINCIPAL Y PLOTEO
# -------------------------------------------------------------

def plot_pie_chart(pie_df: pd.DataFrame, palette: List[str]):
    """Genera y muestra el gráfico de pastel de Plotly."""

    if pie_df.empty:
        st.warning("No hay datos de participación válidos para mostrar el gráfico.")
        return

    # Etiquetas y posición del texto
    pie_df["label_pct"] = pie_df["Part Ene-Sep 2025"].apply(lambda x: es_percent(x, dec=1))
    text_positions = ["outside" if v < 0.006 else "inside" for v in pie_df["Part Ene-Sep 2025"]]

    # Colores
    color_map = build_color_map(pie_df["Aerolínea"], palette)
    colors = [color_map[n] for n in pie_df["Aerolínea"]]

    fig = go.Figure()

    fig.add_trace(
        go.Pie(
            labels=pie_df["Aerolínea"],
            values=pie_df["Part Ene-Sep 2025"],
            text=pie_df["label_pct"],
            textinfo="text",
            textposition=text_positions,
            textfont=dict(family=active_font, size=18),
            marker=dict(colors=colors, line=dict(color="white", width=2)),
            pull=[0.02] * len(pie_df),
            hovertemplate="%{label}<br>Participación: %{percent:.1%}<extra></extra>",
            sort=False,
            direction="clockwise",
            hole=0
        )
    )

    # Layout: Título centrado, leyenda superior derecha
    fig.update_layout(
        title=dict(
            text=TITLE_TEXT,
            x=0.5,             # <--- CENTRADO
            y=0.95,
            xanchor="center",
            yanchor="top"
        ),
        font=dict(family=active_font, size=16),
        width=1100,
        height=680,
        margin=dict(l=50, r=20, t=80, b=50), # Ajustar margen para leyenda
        legend=dict(
            title=None,
            orientation="v",
            x=1.02,
            xanchor="left",
            y=0.82,
            yanchor="top",
            font=dict(family=active_font, size=14),
            itemwidth=30
        ),
    )

    # Reubicar el dominio del pastel (ligeramente a la izquierda para hacer espacio a la leyenda)
    if fig.data:
        fig.data[0].domain = dict(x=[0.0, 0.78], y=[0.0, 1.0])

    st.plotly_chart(fig, use_container_width=True)


if __name__ == "__main__":
    st.markdown("### ✈️ Participación de Mercado Aéreo Nacional")

    try:
        pie_data = load_and_parse_data(DOWNLOAD_URL)
        plot_pie_chart(pie_data, palette)

        with st.expander("Ver tabla de datos"):
            st.dataframe(pie_data, use_container_width=True)
            st.caption("Fuente: AFAC / DataTur (Cálculos propios: Participación Ene-Sep 2025).")

    except RuntimeError as e:
        # Captura errores de parsing y de archivos no encontrados dentro del ZIP
        st.error(f"❌ Error al cargar o parsear los datos: {e}")
    except requests.exceptions.HTTPError as e:
        # Captura errores de descarga (404, 500, etc.)
        st.error(f"❌ Error de descarga: El enlace a DataTur no está disponible. {e}")
    except Exception as e:
        # Captura cualquier otro error inesperado
        st.error(f"❌ Error inesperado: {e}")

Buscando archivo. Si no existe, intentando descargar y extraer desde: https://datatur.sectur.gob.mx/Documentos%20compartidos/CUADRO_DGAC.zip...
✅ Archivo 'AFAC Aerolíneas_SEP_2025.xlsx' extraído y guardado como: AFAC Aerolíneas_SEP_2025.xlsx
Instalando 'kaleido' para exportar PNG...
Error al exportar PNG: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.


